In [1]:
!pip install datasets langchain langchain_openai langchain_community langgraph langchain-google-community sentence-transformers faiss-cpu

# Part 2: All Imports and API Key Setup

In [ ]:
import os
import re
import json
import operator
from google.colab import userdata
from typing import TypedDict, Annotated, List
from datasets import load_dataset
from langchain.tools import tool
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_google_community.search import GoogleSearchAPIWrapper
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, messages_to_dict, messages_from_dict
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode

# Securely load API keys from Colab secrets
try:
    NVIDIA_API_KEY = userdata.get('NVIDIA_API_KEY')
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    GOOGLE_CSE_ID = userdata.get('GOOGLE_CSE_ID')
    print("API Keys loaded successfully.")
except userdata.SecretNotFoundError:
    print("One or more API keys not found in Colab Secrets. Please ensure you have set them up.")
    raise SystemExit("API keys are required to run this script.")


API Keys loaded successfully.


# Part 3: Data Loading and Vector Store Creation

In [10]:
FAISS_INDEX_PATH = "faiss_index_scientific_papers"
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name, show_progress=True)

def get_vector_store():
    """Creates or loads the FAISS vector store."""
    if os.path.exists(FAISS_INDEX_PATH):
        print("Loading existing FAISS vector store...")
        # Allow dangerous deserialization as we are sure of the source
        vector_store = FAISS.load_local(FAISS_INDEX_PATH, embeddings, allow_dangerous_deserialization=True)
        print("Vector store loaded.")
    else:
        print("Creating new FAISS vector store...")
        print("Loading dataset...")
        full_dataset = load_dataset("franz96521/scientific_papers", split='train', streaming=True)
        subset_dataset_iterable = full_dataset.take(100)  # Using 100 papers for the demo
        papers_data = list(subset_dataset_iterable)

        print("Chunking documents...")
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
        all_chunks = []

        for paper in papers_data:
            chunks = text_splitter.split_text(paper['full_text'])
            for chunk in chunks:
                doc = Document(page_content=chunk, metadata={"paper_id": paper['id']})
                all_chunks.append(doc)

        print(f"Embedding {len(all_chunks)} chunks...")
        vector_store = FAISS.from_documents(all_chunks, embeddings)

        print("Saving vector store locally...")
        vector_store.save_local(FAISS_INDEX_PATH)
        print("Vector store created and saved.")

    return vector_store

# Initialize the vector store
vector_store = get_vector_store()

Creating new FAISS vector store...
Loading dataset...
Chunking documents...
Embedding 5516 chunks...


Batches:   0%|          | 0/173 [00:00<?, ?it/s]

Saving vector store locally...
Vector store created and saved.


# Part 4: Tool Definitions

In [23]:
@tool
def paper_qa_tool(query: str) -> str:
    """
    Answers specific, detailed questions about scientific papers on graph theory,
    sparsity, and the pebble game. Use this for questions that reference specific
    paper details or concepts.
    """
    print("--- Calling Paper Q&A Tool ---")
    retriever = vector_store.as_retriever(search_kwargs={'k': 3})
    context_docs = retriever.get_relevant_documents(query)
    gibberish_pattern = re.compile(r'/DAN <[A-Fa-f0-9]+>')
    cleaned_docs = [doc for doc in context_docs if not gibberish_pattern.search(doc.page_content)]

    if not cleaned_docs:
        return "No relevant information found in the documents after cleaning."

    context_text = "\n\n".join([doc.page_content for doc in cleaned_docs])
    return context_text

# Initialize the search wrapper
search_wrapper = GoogleSearchAPIWrapper(google_api_key=GOOGLE_API_KEY, google_cse_id=GOOGLE_CSE_ID)

@tool
def web_search_tool(query: str) -> str:
    """
    Provides up-to-date answers from the web for general knowledge, definitions,
    or topics not covered in the local scientific papers. Also provides source links.
    """
    print("--- Calling Web Search Tool ---")
    results = search_wrapper.results(query, num_results=3)
    return "\n".join([f"Title: {res['title']}\nLink: {res['link']}\nSnippet: {res['snippet']}\n" for res in results])

# Create tools list
tools = [paper_qa_tool, web_search_tool]

# Use ToolNode instead of ToolExecutor (modern approach)
tool_node = ToolNode(tools)

# Part 5: LangGraph Agent Definition

In [26]:
from typing import TypedDict, List
from typing_extensions import Annotated
import operator
from openai import OpenAI

# Your NVIDIA API client
client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=NVIDIA_API_KEY
)

# Define the AgentState
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]

# Remove ChatOpenAI and use the OpenAI client directly
def call_model(state):
    print("--- AGENT: Thinking... ---")

    # Convert your BaseMessage objects to OpenAI-compatible messages
    last_messages = [
        {"role": "user" if m.type == "human" else "assistant", "content": m.content}
        for m in state['messages']
    ]
    print(last_messages)

    # Call the NVIDIA OpenAI client
    completion = client.chat.completions.create(
        model="openai/gpt-oss-120b",
        messages=last_messages,
        temperature=0.2,
        top_p=1,
        max_tokens=4096
    )

    response_text = completion.choices[0].message.content
    # print(response_text)

    # Use AIMessage instead of BaseMessage - this includes the required 'type' field
    from langchain_core.messages import AIMessage
    response_message = AIMessage(content=response_text)
    return {"messages": [response_message]}

# Tool execution stays the same
def call_tool(state):
    last_message = state['messages'][-1]
    tool_invocations = []
    for tool_call in last_message.tool_calls:
        action = ToolInvocation(tool=tool_call["name"], tool_input=tool_call["args"])
        tool_invocations.append(action)

    responses = tool_executor.batch(tool_invocations, return_exceptions=True)
    tool_messages = [
        ToolMessage(content=str(res), name=inv.tool, tool_call_id=call["id"])
        for res, inv, call in zip(responses, tool_invocations, last_message.tool_calls)
    ]
    return {"messages": tool_messages}

def should_continue(state):
    if not state['messages'][-1].tool_calls:
        return "end"
    else:
        return "continue"

# Build the workflow
workflow = StateGraph(AgentState)
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)
workflow.set_entry_point("agent")
workflow.add_conditional_edges("agent", should_continue, {"continue": "action", "end": END})
workflow.add_edge("action", "agent")

app = workflow.compile()
print("\nResearch Agent is ready!")



Research Agent is ready!


# Part 6: Local Chat History and Conversational Loop

In [30]:
def save_history(messages, file_path="chat_history.json"):
    dict_messages = messages_to_dict(messages)
    with open(file_path, "w") as f:
        json.dump(dict_messages, f, indent=2)

def load_history(file_path="chat_history.json"):
    if os.path.exists(file_path):
        with open(file_path, "r") as f:
            dict_messages = json.load(f)
        return messages_from_dict(dict_messages)
    return []

messages = load_history()
print("\nType 'exit' to end the conversation.")
if messages:
    print("--- Resuming previous conversation ---")
    for msg in messages:
        if isinstance(msg, HumanMessage):
            print(f"USER: {msg.content}")
        elif isinstance(msg, AIMessage):
            print(f"AGENT: {msg.content}")
    print("------------------------------------")

while True:
    query = input("USER: ")
    if query.lower() == 'exit':
        print("AGENT: Goodbye!")
        break

    messages.append(HumanMessage(content=query))

    print("AGENT: ", end="", flush=True)
    full_response = ""
    for event in app.stream({"messages": messages}):
        for value in event.values():
            if isinstance(value["messages"][-1], AIMessage) and value["messages"][-1].content:
                print(value["messages"][-1].content, end="", flush=True)
                full_response += value["messages"][-1].content
    print("\n")

    if full_response:
        messages.append(AIMessage(content=full_response))
        save_history(messages)


Type 'exit' to end the conversation.
USER: What is a sparsity-certifying decomposition according to the provided papers?
AGENT: --- AGENT: Thinking... ---
[{'role': 'user', 'content': 'What is a sparsity-certifying decomposition according to the provided papers?'}]
Below is a concise synthesis of how the papers you were given define and use a **sparsity‑certifying decomposition**.  I have grouped the description into (i) the formal definition, (ii) why it is called “certifying”, (iii) the typical algebraic form it takes in the different settings considered in the papers, and (iv) how the authors construct it and what guarantees it yields.

---

## 1.  Formal definition (common to all papers)

A **sparsity‑certifying decomposition** is a *pair of objects* (usually a primal object and a dual object) that together satisfy the optimality conditions of a sparsity‑promoting convex program **and** explicitly exhibit the sparsity pattern that the program is supposed to recover.  

In other wo